# Functional Lesion Network Mapping (fLNM) - Comprehensive Tutorial

This notebook demonstrates all the functionality of the **Functional Lesion Network Mapping** (fLNM) analysis in the Lesion Decoding Toolkit.

## What is fLNM?

Functional Lesion Network Mapping identifies brain-wide functional connectivity disruptions associated with a focal lesion by:
1. Extracting the lesion's timeseries from a normative connectome
2. Computing correlations with all other brain voxels
3. Producing a **correlation map** (r-map) showing connectivity strength
4. Optionally computing **t-statistic maps** and **binary threshold maps**

## What You'll Learn

1. ✅ Basic single-subject analysis
2. ✅ Both methods: BOES and PINI
3. ✅ Batch processing multiple lesions
4. ✅ Memory-efficient processing with large connectomes
5. ✅ Computing t-statistics and thresholding
6. ✅ Interpreting and visualizing results
7. ✅ Performance optimization tips

Let's get started! 🚀

## Setup: Import Libraries and Create Test Data

First, we'll import the necessary libraries and create some mock data for demonstration.

In [ ]:
# Standard imports
import numpy as np
import nibabel as nib
import h5py
from pathlib import Path
import tempfile
import matplotlib.pyplot as plt

# LDK imports
from lacuna import LesionData
from lacuna.analysis import FunctionalNetworkMapping
from lacuna.batch import batch_process

print("✓ All imports successful!")

### Create Mock Connectome Data

We'll create a realistic mock connectome with:
- 100 subjects
- 100 timepoints per subject
- 50,000 voxels (typical for 2mm MNI152 space)
- Split into multiple batches to demonstrate batch processing

In [ ]:
# Create temporary directory for our demo
demo_dir = Path(tempfile.mkdtemp(prefix="flnm_demo_"))
connectome_dir = demo_dir / "connectome_batches"
connectome_dir.mkdir()
lesion_dir = demo_dir / "lesions"
lesion_dir.mkdir()
output_dir = demo_dir / "results"
output_dir.mkdir()

print(f"Demo directory: {demo_dir}")
print(f"Connectome directory: {connectome_dir}")
print(f"Lesion directory: {lesion_dir}")
print(f"Output directory: {output_dir}")

In [ ]:
# Create realistic connectome parameters
np.random.seed(42)

n_voxels = 10000  # Reduced for demo speed (typical: 50,000)
n_timepoints = 100
n_subjects_per_batch = 20
n_batches = 5  # Total: 100 subjects

# Create realistic brain mask indices (3D coordinates)
# Simulate voxels distributed across the brain
x_coords = np.random.randint(10, 80, size=n_voxels)
y_coords = np.random.randint(10, 99, size=n_voxels)
z_coords = np.random.randint(10, 80, size=n_voxels)
mask_indices = np.array([x_coords, y_coords, z_coords])

# MNI152 2mm affine matrix
mask_affine = np.array(
    [[-2.0, 0.0, 0.0, 90.0], [0.0, 2.0, 0.0, -126.0], [0.0, 0.0, 2.0, -72.0], [0.0, 0.0, 0.0, 1.0]]
)

mask_shape = (91, 109, 91)

print(f"Creating {n_batches} connectome batches...")
print(f"  - {n_subjects_per_batch} subjects per batch")
print(f"  - {n_timepoints} timepoints")
print(f"  - {n_voxels} voxels")

# Create connectome batch files
for batch_idx in range(n_batches):
    batch_path = connectome_dir / f"batch_{batch_idx:03d}.h5"

    # Generate realistic timeseries with some structure
    # Add correlated noise to simulate real fMRI data
    base_signal = np.random.randn(n_timepoints, n_voxels)
    timeseries = np.zeros((n_subjects_per_batch, n_timepoints, n_voxels))

    for subj in range(n_subjects_per_batch):
        # Each subject gets base signal + individual variation
        subj_signal = base_signal + np.random.randn(n_timepoints, n_voxels) * 0.5
        timeseries[subj] = subj_signal

    # Save to HDF5
    with h5py.File(batch_path, "w") as f:
        f.create_dataset("timeseries", data=timeseries.astype(np.float32))
        f.create_dataset("mask_indices", data=mask_indices)
        f.create_dataset("mask_affine", data=mask_affine)
        f.attrs["mask_shape"] = mask_shape

    print(f"  ✓ Created batch {batch_idx + 1}/{n_batches}")

print(f"\n✅ Connectome data created successfully!")

### Create Mock Lesions

Now let's create several lesions with different sizes and locations to demonstrate the analysis.

In [ ]:
# Create multiple lesions for testing
lesion_specs = [
    {"name": "small_frontal", "location": (45, 65, 45), "size": 3},
    {"name": "medium_parietal", "location": (35, 50, 55), "size": 5},
    {"name": "large_temporal", "location": (55, 45, 35), "size": 7},
]

lesions = []

for spec in lesion_specs:
    # Create lesion volume
    lesion_data = np.zeros(mask_shape, dtype=np.uint8)

    x, y, z = spec["location"]
    s = spec["size"]

    # Create cubic lesion
    lesion_data[x - s : x + s, y - s : y + s, z - s : z + s] = 1

    # Create NIfTI image
    lesion_img = nib.Nifti1Image(lesion_data, mask_affine)

    # Save to file
    lesion_path = lesion_dir / f"{spec['name']}.nii.gz"
    nib.save(lesion_img, lesion_path)

    # Load with LesionData
    lesion = LesionData.from_nifti(
        str(lesion_path),
        metadata={
            "space": "MNI152_2mm",
            "subject_id": spec["name"],
            "lesion_volume_mm3": int(np.sum(lesion_data) * 8),  # 2mm^3 voxels
        },
    )
    lesions.append(lesion)

    print(f"✓ Created lesion: {spec['name']}")
    print(f"    Location: {spec['location']}, Size: {spec['size']}")
    print(f"    Volume: {lesion.metadata['lesion_volume_mm3']} mm³")

print(f"\n✅ Created {len(lesions)} lesions for testing")

---
## Example 1: Basic Single-Subject Analysis (BOES Method)

The simplest use case: analyze one lesion using the **BOES method** (mean timeseries across all lesion voxels).

In [ ]:
# Create analysis object
analysis_boes = FunctionalNetworkMapping(
    connectome_path=str(connectome_dir),
    method="boes",  # Mean timeseries (default)
    verbose=True,  # Show progress
    compute_t_map=False,  # Skip t-statistics for now
)

print("Analysis Configuration:")
print(f"  Method: {analysis_boes.method}")
print(f"  Connectome: {analysis_boes.connectome_path}")
print(f"  Batch strategy: {analysis_boes.batch_strategy}")

In [ ]:
# Run analysis on single lesion
print("\n" + "=" * 70)
print("RUNNING SINGLE-SUBJECT ANALYSIS")
print("=" * 70 + "\n")

result = analysis_boes.run(lesions[0])

print("\n✅ Analysis complete!")
print(f"Results available in: {result.results.keys()}")

### Inspect Results

Let's look at what the analysis produced:

In [ ]:
# Get fLNM results
flnm_results = result.results["FunctionalNetworkMapping"]

print("Available outputs:")
for key in flnm_results.keys():
    value = flnm_results[key]
    if isinstance(value, nib.Nifti1Image):
        print(f"  • {key}: NIfTI image, shape {value.shape}")
    elif isinstance(value, dict):
        print(f"  • {key}: Dictionary with {len(value)} entries")
    else:
        print(f"  • {key}: {type(value).__name__}")

print("\nSummary Statistics:")
for key, value in flnm_results["summary_statistics"].items():
    print(f"  • {key}: {value}")

### Visualize Correlation Map

Let's visualize the functional connectivity pattern:

In [ ]:
# Extract correlation map
correlation_map = flnm_results["correlation_map"].get_fdata()

# Plot 3 orthogonal slices
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Axial slice (z)
z_slice = 45
im1 = axes[0].imshow(
    correlation_map[:, :, z_slice].T, cmap="RdBu_r", vmin=-0.5, vmax=0.5, origin="lower"
)
axes[0].set_title(f"Axial Slice (z={z_slice})")
axes[0].axis("off")

# Coronal slice (y)
y_slice = 54
im2 = axes[1].imshow(
    correlation_map[:, y_slice, :].T, cmap="RdBu_r", vmin=-0.5, vmax=0.5, origin="lower"
)
axes[1].set_title(f"Coronal Slice (y={y_slice})")
axes[1].axis("off")

# Sagittal slice (x)
x_slice = 45
im3 = axes[2].imshow(
    correlation_map[x_slice, :, :].T, cmap="RdBu_r", vmin=-0.5, vmax=0.5, origin="lower"
)
axes[2].set_title(f"Sagittal Slice (x={x_slice})")
axes[2].axis("off")

# Add colorbar
plt.colorbar(
    im1, ax=axes, orientation="horizontal", label="Correlation (r)", fraction=0.046, pad=0.04
)
plt.suptitle(
    f"Functional Connectivity Map: {lesions[0].metadata['subject_id']}",
    fontsize=14,
    fontweight="bold",
)
plt.tight_layout()
plt.show()

print(f"Mean correlation: {flnm_results['mean_correlation']:.4f}")
print(f"Correlation range: [{correlation_map.min():.4f}, {correlation_map.max():.4f}]")

---
## Example 2: PINI Method (PCA-based Voxel Selection)

The **PINI method** uses PCA to select the most representative lesion voxels, which can improve signal-to-noise ratio.

In [ ]:
# Create analysis with PINI method
analysis_pini = FunctionalNetworkMapping(
    connectome_path=str(connectome_dir),
    method="pini",  # PCA-based selection
    pini_percentile=75,  # Use top 25% of voxels
    verbose=True,
    compute_t_map=False,
)

print("PINI Configuration:")
print(f"  Method: {analysis_pini.method}")
print(f"  Percentile threshold: {analysis_pini.pini_percentile}")

In [ ]:
# Run PINI analysis on same lesion
print("\n" + "=" * 70)
print("RUNNING PINI ANALYSIS")
print("=" * 70 + "\n")

result_pini = analysis_pini.run(lesions[0])

# Compare with BOES
flnm_pini = result_pini.results["FunctionalNetworkMapping"]

print("\nComparison: BOES vs PINI")
print(f"  BOES mean correlation: {flnm_results['mean_correlation']:.4f}")
print(f"  PINI mean correlation: {flnm_pini['mean_correlation']:.4f}")
print(f"  Difference: {abs(flnm_pini['mean_correlation'] - flnm_results['mean_correlation']):.4f}")

---
## Example 3: Computing T-Statistics and Threshold Maps

For group-level inference, we can compute **t-statistic maps** to identify significantly connected regions.

In [ ]:
# Create analysis with t-statistics
analysis_with_t = FunctionalNetworkMapping(
    connectome_path=str(connectome_dir),
    method="boes",
    verbose=True,
    compute_t_map=True,  # Compute t-statistics
    t_threshold=2.5,  # Threshold at |t| > 2.5
)

print("Configuration with t-statistics:")
print(f"  Compute t-map: {analysis_with_t.compute_t_map}")
print(f"  t-threshold: {analysis_with_t.t_threshold}")

In [ ]:
# Run analysis
result_t = analysis_with_t.run(lesions[1])  # Use medium-sized lesion

flnm_t = result_t.results["FunctionalNetworkMapping"]

print("\nResults with t-statistics:")
print(f"  • t_map: shape {flnm_t['t_map'].shape}")
print(f"  • t_threshold_map: shape {flnm_t['t_threshold_map'].shape}")

print("\nT-statistic summary:")
print(f"  • t-min: {flnm_t['summary_statistics']['t_min']:.2f}")
print(f"  • t-max: {flnm_t['summary_statistics']['t_max']:.2f}")
print(f"  • Significant voxels (|t| > 2.5): {flnm_t['summary_statistics']['n_significant_voxels']}")

### Visualize T-Statistics


In [ ]:
# Extract maps
t_map = flnm_t["t_map"].get_fdata()
threshold_map = flnm_t["t_threshold_map"].get_fdata()

# Plot
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# T-map slices
z_slice = 45
axes[0, 0].imshow(t_map[:, :, z_slice].T, cmap="RdBu_r", vmin=-5, vmax=5, origin="lower")
axes[0, 0].set_title(f"T-map: Axial (z={z_slice})")
axes[0, 0].axis("off")

y_slice = 54
axes[0, 1].imshow(t_map[:, y_slice, :].T, cmap="RdBu_r", vmin=-5, vmax=5, origin="lower")
axes[0, 1].set_title(f"T-map: Coronal (y={y_slice})")
axes[0, 1].axis("off")

x_slice = 45
axes[0, 2].imshow(t_map[x_slice, :, :].T, cmap="RdBu_r", vmin=-5, vmax=5, origin="lower")
axes[0, 2].set_title(f"T-map: Sagittal (x={x_slice})")
axes[0, 2].axis("off")

# Threshold map slices
axes[1, 0].imshow(threshold_map[:, :, z_slice].T, cmap="hot", origin="lower")
axes[1, 0].set_title(f"Significant Voxels: Axial")
axes[1, 0].axis("off")

axes[1, 1].imshow(threshold_map[:, y_slice, :].T, cmap="hot", origin="lower")
axes[1, 1].set_title(f"Significant Voxels: Coronal")
axes[1, 1].axis("off")

axes[1, 2].imshow(threshold_map[x_slice, :, :].T, cmap="hot", origin="lower")
axes[1, 2].set_title(f"Significant Voxels: Sagittal")
axes[1, 2].axis("off")

plt.suptitle(f"T-Statistics: {result_t.metadata['subject_id']}", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---
## Example 4: Batch Processing Multiple Lesions

Process all lesions efficiently using **vectorized batch processing** - the recommended method for multiple subjects.

In [ ]:
# Create analysis for batch processing
analysis_batch = FunctionalNetworkMapping(
    connectome_path=str(connectome_dir),
    method="boes",
    verbose=True,
    compute_t_map=True,
    t_threshold=2.0,
)

print("Batch Processing Configuration:")
print(f"  Number of lesions: {len(lesions)}")
print(f"  Strategy: vectorized (automatic)")
print(f"  Compute t-maps: Yes")

In [ ]:
# Process all lesions at once
print("\n" + "=" * 70)
print("BATCH PROCESSING ALL LESIONS")
print("=" * 70 + "\n")

results_batch = batch_process(
    lesions,
    analysis_batch,
    strategy="vectorized",  # Vectorized = fastest for multiple lesions
    show_progress=True,
)

print(f"\n✅ Processed {len(results_batch)} lesions successfully!")

### Compare Results Across Lesions

In [ ]:
# Create comparison table
import pandas as pd

comparison_data = []
for result in results_batch:
    flnm = result.results["FunctionalNetworkMapping"]
    stats = flnm["summary_statistics"]

    comparison_data.append(
        {
            "Subject": result.metadata["subject_id"],
            "Mean r": stats["mean"],
            "Std r": stats["std"],
            "Max r": stats["max"],
            "Min r": stats["min"],
            "t-max": stats.get("t_max", np.nan),
            "t-min": stats.get("t_min", np.nan),
            "Sig. voxels": stats.get("n_significant_voxels", 0),
        }
    )

df = pd.DataFrame(comparison_data)
print("\nBatch Processing Results:")
print(df.to_string(index=False))

---
## Example 5: Memory-Efficient Processing with Lesion Batching

For very large numbers of lesions, process them in smaller batches to control memory usage.

In [ ]:
# Define callback to save results incrementally
def save_batch_results(batch_results):
    """Save results after each batch to free memory."""
    for result in batch_results:
        subject_id = result.metadata["subject_id"]
        flnm = result.results["FunctionalNetworkMapping"]

        # Save correlation map
        rmap_path = output_dir / f"{subject_id}_rmap.nii.gz"
        nib.save(flnm["correlation_map"], rmap_path)

        # Save t-map if available
        if "t_map" in flnm:
            tmap_path = output_dir / f"{subject_id}_tmap.nii.gz"
            nib.save(flnm["t_map"], tmap_path)

        print(f"    💾 Saved: {subject_id}")


print("Memory-Efficient Batch Processing:")
print(f"  Lesion batch size: 2")
print(f"  Total lesions: {len(lesions)}")
print(f"  Expected batches: {(len(lesions) + 1) // 2}")

In [ ]:
# Process with memory control
print("\n" + "=" * 70)
print("MEMORY-EFFICIENT BATCH PROCESSING")
print("=" * 70 + "\n")

results_memory_efficient = batch_process(
    lesions,
    analysis_batch,
    strategy="vectorized",
    lesion_batch_size=2,  # Process 2 lesions at a time
    batch_result_callback=save_batch_results,  # Save immediately
    show_progress=True,
)

print(f"\n✅ All results saved to: {output_dir}")
print(f"✅ Files created:")
for file in sorted(output_dir.glob("*.nii.gz")):
    print(f"    • {file.name}")

---
## Example 6: Performance Monitoring and Timing

Monitor batch processing performance to optimize your workflow.

In [ ]:
import time

# Time the processing
print("Performance Test: Processing 3 lesions")
print("-" * 50)

start_time = time.time()

results_timed = batch_process(
    lesions,
    analysis_batch,
    strategy="vectorized",
    show_progress=False,  # Reduce output for timing
)

total_time = time.time() - start_time

print(f"\nPerformance Results:")
print(f"  Total time: {total_time:.2f}s")
print(f"  Time per lesion: {total_time / len(lesions):.2f}s")
print(f"  Total subjects processed: {100}")
print(f"  Time per subject: {total_time / 100:.3f}s")

---
## Example 7: Saving and Loading Results

Learn how to save your analysis results and load them later.

In [ ]:
# Save all outputs for a lesion
subject_id = lesions[0].metadata["subject_id"]
result_to_save = results_batch[0]
flnm_to_save = result_to_save.results["FunctionalNetworkMapping"]

print(f"Saving results for: {subject_id}")

# Save all NIfTI outputs
output_files = []

# Correlation map
rmap_file = output_dir / f"{subject_id}_correlation_map.nii.gz"
nib.save(flnm_to_save["correlation_map"], rmap_file)
output_files.append(rmap_file)

# Z-map
zmap_file = output_dir / f"{subject_id}_z_map.nii.gz"
nib.save(flnm_to_save["z_map"], zmap_file)
output_files.append(zmap_file)

# T-map (if computed)
if "t_map" in flnm_to_save:
    tmap_file = output_dir / f"{subject_id}_t_map.nii.gz"
    nib.save(flnm_to_save["t_map"], tmap_file)
    output_files.append(tmap_file)

# Threshold map (if computed)
if "t_threshold_map" in flnm_to_save:
    threshmap_file = output_dir / f"{subject_id}_threshold_map.nii.gz"
    nib.save(flnm_to_save["t_threshold_map"], threshmap_file)
    output_files.append(threshmap_file)

print(f"\n✅ Saved {len(output_files)} files:")
for file in output_files:
    print(f"    • {file.name}")

In [ ]:
# Load results back
print("\nLoading results...")

loaded_rmap = nib.load(str(rmap_file))
loaded_zmap = nib.load(str(zmap_file))

print(f"  ✓ Loaded correlation map: {loaded_rmap.shape}")
print(f"  ✓ Loaded z-map: {loaded_zmap.shape}")

# Verify data integrity
original_rmap_data = flnm_to_save["correlation_map"].get_fdata()
loaded_rmap_data = loaded_rmap.get_fdata()

match = np.allclose(original_rmap_data, loaded_rmap_data, rtol=1e-5)
print(f"\n✅ Data integrity check: {'PASSED' if match else 'FAILED'}")

---
## Summary: Key Takeaways

### Methods Available
1. **BOES** (default): Mean timeseries across all lesion voxels
   - Fast, straightforward
   - Good for most use cases
   
2. **PINI**: PCA-based voxel selection
   - Can improve signal-to-noise
   - Use `pini_percentile` to control selection

### Processing Strategies
1. **Single subject**: Use `analysis.run(lesion)`
2. **Multiple subjects**: Use `batch_process(lesions, analysis)`
3. **Memory-constrained**: Use `lesion_batch_size` parameter

### Outputs Generated
- **correlation_map**: r-values showing connectivity strength
- **z_map**: Fisher z-transformed correlations
- **t_map**: t-statistics for group inference (optional)
- **t_threshold_map**: Binary map of significant voxels (optional)
- **summary_statistics**: Mean, std, max, min, etc.

### Performance Tips
- ✅ Use **vectorized strategy** for multiple lesions (10-50x faster)
- ✅ Set `lesion_batch_size` to control memory (recommended: 20-50)
- ✅ Use `batch_result_callback` for incremental saving
- ✅ Split large connectomes into multiple HDF5 files
- ✅ Monitor timing with `verbose=True`

### Next Steps
- Try with your own lesion data
- Experiment with different parameters
- Compare BOES vs PINI for your use case
- Visualize results with your preferred tools

---
## Cleanup

Clean up the temporary files created during this demo.

In [ ]:
# Cleanup (optional - comment out to keep files)
import shutil

# shutil.rmtree(demo_dir)
# print(f"✓ Cleaned up demo directory: {demo_dir}")

print(f"\nDemo files preserved at: {demo_dir}")
print("To remove them, uncomment the cleanup code above.")

---
## Additional Resources

### Documentation
- **User Guide**: `docs/analysis/functional_network_mapping.md`
- **API Reference**: `docs/api/analysis.md`
- **Memory Management**: `docs/memory_management.md`
- **Batch Processing**: `docs/batch_timing.md`

### Example Scripts
- **Batch processing**: `examples/batch_processing_example.py`
- **Production script**: `test_batch_flnm.py`

### Getting Help
- **GitHub Issues**: Report bugs or request features
- **Discussions**: Ask questions and share experiences

Happy analyzing! 🧠✨